In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import collections
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
import re
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 39.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=efe1a696a69fec448f5949871b168f59abaa8d0e28b9d75b4681868713df5f7a
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("PySpark Read JSON") \
    .getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#df = spark.read.json("gs://kyaw-spark/notebooks/jupyter/train-v2.0.json")#
df = spark.read.json("/content/drive/MyDrive/Colab Notebooks/NLP_codealong/train-v1.1.json")


In [7]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- paragraphs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- context: string (nullable = true)
 |    |    |    |    |-- qas: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |    |-- question: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- version: string (nullable = true)



In [8]:
df.show(5)

+--------------------+-------+
|                data|version|
+--------------------+-------+
|[{[{Architectural...|    1.1|
+--------------------+-------+



In [9]:
from pyspark.sql.functions import col, explode, split, json_tuple, concat_ws, lit, udf

In [10]:
df_explode =  df.select(explode(col('data')).alias('data')).select("data.paragraphs", "data.title")

In [11]:
df_explode.printSchema()

root
 |-- paragraphs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- context: string (nullable = true)
 |    |    |-- qas: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- question: string (nullable = true)
 |-- title: string (nullable = true)



In [12]:
df_nd = df_explode.filter("title == 'University_of_Notre_Dame'")

In [13]:
df_nd.show()

+--------------------+--------------------+
|          paragraphs|               title|
+--------------------+--------------------+
|[{Architecturally...|University_of_Not...|
+--------------------+--------------------+



In [14]:
df_nd.printSchema()

root
 |-- paragraphs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- context: string (nullable = true)
 |    |    |-- qas: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- question: string (nullable = true)
 |-- title: string (nullable = true)



In [15]:
df_nd.createOrReplaceTempView('result_1')

In [16]:
subset_nd = spark.sql(
'''
SELECT paragraphs.qas, paragraphs.context
from result_1
'''
).persist()

In [17]:
subset_nd.printSchema()

root
 |-- qas: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- answers: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- question: string (nullable = true)
 |-- context: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [18]:
subset_nd.show()

+--------------------+--------------------+
|                 qas|             context|
+--------------------+--------------------+
|[[{[{515, Saint B...|[Architecturally,...|
+--------------------+--------------------+



In [19]:
nd_context = subset_nd.select("context", explode(col('qas')).alias('qas'))

In [20]:
nd_context.printSchema()

root
 |-- context: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qas: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- answers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- answer_start: long (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- question: string (nullable = true)



In [21]:
nd_context.show(1)

+--------------------+--------------------+
|             context|                 qas|
+--------------------+--------------------+
|[Architecturally,...|[{[{515, Saint Be...|
+--------------------+--------------------+
only showing top 1 row



In [22]:
nd_question_answer = nd_context.select("context", explode(col('qas')).alias('x')).select("context","x.answers","x.question")

In [23]:
nd_question_answer.printSchema()

root
 |-- context: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- answers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- answer_start: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- question: string (nullable = true)



In [24]:
nd_question_answer.show(5)

+--------------------+--------------------+--------------------+
|             context|             answers|            question|
+--------------------+--------------------+--------------------+
|[Architecturally,...|[{515, Saint Bern...|To whom did the V...|
|[Architecturally,...|[{188, a copper s...|What is in front ...|
|[Architecturally,...|[{279, the Main B...|The Basilica of t...|
|[Architecturally,...|[{381, a Marian p...|What is the Grott...|
|[Architecturally,...|[{92, a golden st...|What sits on top ...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [25]:
nd_question_answer.printSchema()

root
 |-- context: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- answers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- answer_start: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- question: string (nullable = true)



In [26]:
nd_question_answer.show()

+--------------------+--------------------+--------------------+
|             context|             answers|            question|
+--------------------+--------------------+--------------------+
|[Architecturally,...|[{515, Saint Bern...|To whom did the V...|
|[Architecturally,...|[{188, a copper s...|What is in front ...|
|[Architecturally,...|[{279, the Main B...|The Basilica of t...|
|[Architecturally,...|[{381, a Marian p...|What is the Grott...|
|[Architecturally,...|[{92, a golden st...|What sits on top ...|
|[Architecturally,...|[{248, September ...|When did the Scho...|
|[Architecturally,...|      [{441, twice}]|How often is Notr...|
|[Architecturally,...|[{598, The Observ...|What is the daily...|
|[Architecturally,...|      [{126, three}]|How many student ...|
|[Architecturally,...|       [{908, 1987}]|In what year did ...|
|[Architecturally,...|       [{119, Rome}]|Where is the head...|
|[Architecturally,...|[{145, Moreau Sem...|What is the prima...|
|[Architecturally,...|[{2

In [27]:
df_context1 =   nd_question_answer.select("context")
df_d =df_context1.dropDuplicates()

In [28]:
nd_question = nd_question_answer.select("context","question",explode(col('answers')).alias('x')).select("context","question", "x.text")

In [29]:
nd_question.count()

269

In [30]:
df_final = nd_question.withColumn("context",concat_ws(",",col("context"))) \
.withColumn("question",concat_ws(",",col("question"))) \
.withColumn("text",concat_ws(",",col("text")))



In [31]:
df_final.count()

269

In [32]:
df_refine = df_final.select("context","question","text").rdd.map(lambda x: [x[i] for i in [0,1,2]]).toDF()

In [33]:
df_refine.show(5)

+--------------------+--------------------+--------------------+
|                  _1|                  _2|                  _3|
+--------------------+--------------------+--------------------+
|Architecturally, ...|To whom did the V...|Saint Bernadette ...|
|Architecturally, ...|What is in front ...|a copper statue o...|
|Architecturally, ...|The Basilica of t...|   the Main Building|
|Architecturally, ...|What is the Grott...|a Marian place of...|
|Architecturally, ...|What sits on top ...|a golden statue o...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [34]:
data = df_refine.select(col("_1").alias("sentence"), col("_2").alias("question"), col("_3").alias("text"))

In [35]:
data.show(7)

+--------------------+--------------------+--------------------+
|            sentence|            question|                text|
+--------------------+--------------------+--------------------+
|Architecturally, ...|To whom did the V...|Saint Bernadette ...|
|Architecturally, ...|What is in front ...|a copper statue o...|
|Architecturally, ...|The Basilica of t...|   the Main Building|
|Architecturally, ...|What is the Grott...|a Marian place of...|
|Architecturally, ...|What sits on top ...|a golden statue o...|
|Architecturally, ...|When did the Scho...|      September 1876|
|Architecturally, ...|How often is Notr...|               twice|
+--------------------+--------------------+--------------------+
only showing top 7 rows



In [36]:
pred_data = data.toPandas()

In [ ]:
#final.to_csv("/content/drive/MyDrive/Applied Data Science/project_v1", sep='\t')

### Prediction

In [37]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=20c5a608fb308c04fc00b36a033bd527e801d639c052f3e68b3d25a874c2bd3a
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

In [39]:
def preddicted_index_value(distances):
    return np.argmin(distances)

In [40]:
def predict(train):
    
    train["cosine_sim"] = train.apply(cosine_sim_distance_bart, axis = 1)
    train["predicted_index"] = train["cosine_sim"].apply(lambda x:preddicted_index_value(x))
    
    return train

In [41]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.5 MB 32.2 MB/s 
     |████████████████████████████████| 1.3 MB 37.5 MB/s 
     |████████████████████████████████| 182 kB 41.4 MB/s 
     |████████████████████████████████| 7.6 MB 10.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2183bb071ea0cd31ddd99d494364301b9f6ea42c4c30d6ca334d2f8115a0b681
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [42]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
# bert-base-nli-mean-tokens
#all-MiniLM-L6-v2

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [43]:
paras = list(pred_data["sentence"].drop_duplicates().reset_index(drop= True))

In [46]:
questions = list(pred_data["question"])

In [49]:
from textblob import TextBlob
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [87]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [88]:
blob.sentences[0].tags

[('Architecturally', 'RB'),
 ('the', 'DT'),
 ('school', 'NN'),
 ('has', 'VBZ'),
 ('a', 'DT'),
 ('Catholic', 'JJ'),
 ('character', 'NN')]

In [50]:
dict_embeddings = {}
for i in range(len(sentences)):
 
    dict_embeddings[sentences[i]] = sbert_model.encode([sentences[i]])

In [51]:
#dict_embeddings_q = {}
for i in range(len(questions)):
 
    dict_embeddings[questions[i]] = sbert_model.encode([questions[i]])

In [52]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings)}

In [ ]:
#d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings)}

In [257]:
#d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings_q)}

In [53]:
pred_data[0:5]

,sentence,question,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [54]:
pred_data.isna().values.any()

False

In [55]:
def process_data(t):
    
    print("step 1")
    t['sentences'] = t['sentence'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    t["target"] = t.apply(get_target, axis = 1)
    print("step 3")
    t['sent_emb'] = t['sentences'].apply(lambda x: [d1[item][0] if item in\
                                                           d1 else np.zeros(100) for item in x])
    print("step 4")
    t['quest_emb'] = t['question'].apply(lambda x: d1[x] if x in d1 else np.zeros(100) )
   
        
    return t 

In [56]:
pred_data.dropna(inplace=True)

In [57]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [58]:
df_embed = process_data(pred_data)

step 1
step 2
step 3
step 4


In [59]:
df_embed

,sentence,question,text,sentences,target,sent_emb,quest_emb
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.018169405, 0.055049352, -0.00028669654, 0..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.031450234, 0.07019896, 0.042816784, -0.032..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...",248,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.05156063, 0.06941284, 0.018183814, -0.0388..."
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.018691333, 0.05136551, 0.03096037, -0.0819..."
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.017544404, 0.019928047, 0.05257405, -0.042..."
...,...,...,...,...,...,...,...
264,"Architecturally, the school has a Catholic cha...",Which Secretary of State attended Notre Dame?,Condoleezza Rice,"[Architecturally, the school has a Catholic ch...",318,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.013967387, 0.028815048, 0.06493278, -0.02..."
265,"Architecturally, the school has a Catholic cha...",Which Notre Dame alum from the College of Scie...,Eric F. Wieschaus,"[Architecturally, the school has a Catholic ch...",319,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.051872168, 0.005974253, 0.01631587, -0.01..."
266,"Architecturally, the school has a Catholic cha...",Who is the current president of Notre Dame?,Rev. John Jenkins,"[Architecturally, the school has a Catholic ch...",-1,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.023697998, -0.017766533, 0.06430511, -0.0..."
267,"Architecturally, the school has a Catholic cha...",Mariel Zagunis is notable for winning what?,Olympic gold,"[Architecturally, the school has a Catholic ch...",323,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.042852353, 0.070303954, -0.043868415, -0...."


In [274]:
#bart_df.to_csv("/content/drive/MyDrive/Applied Data Science/project_bart_v1", sep='\t')

In [60]:
def cosine_sim_distance_bart(x):
    cosine_list = []
    for item in x["sent_emb"]:
      cosine_list.append(distance.cosine(item, x['quest_emb'][0]))
    return cosine_list   

In [61]:
predict(df_embed)

,sentence,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,predicted_index
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.018169405, 0.055049352, -0.00028669654, 0...","[0.7376718521118164, 0.5794012546539307, 0.692...",5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.031450234, 0.07019896, 0.042816784, -0.032...","[0.48134666681289673, 0.5599262714385986, 0.52...",6
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...",248,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.05156063, 0.06941284, 0.018183814, -0.0388...","[0.5370956063270569, 0.5427329540252686, 0.642...",3
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.018691333, 0.05136551, 0.03096037, -0.0819...","[0.5678138732910156, 0.687394380569458, 0.6089...",168
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[0.017544404, 0.019928047, 0.05257405, -0.042...","[0.4779335856437683, 0.5311467051506042, 0.556...",6
...,...,...,...,...,...,...,...,...,...
264,"Architecturally, the school has a Catholic cha...",Which Secretary of State attended Notre Dame?,Condoleezza Rice,"[Architecturally, the school has a Catholic ch...",318,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.013967387, 0.028815048, 0.06493278, -0.02...","[0.6608383655548096, 0.9387792311608791, 0.899...",320
265,"Architecturally, the school has a Catholic cha...",Which Notre Dame alum from the College of Scie...,Eric F. Wieschaus,"[Architecturally, the school has a Catholic ch...",319,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.051872168, 0.005974253, 0.01631587, -0.01...","[0.6738191545009613, 0.846792921423912, 0.8896...",319
266,"Architecturally, the school has a Catholic cha...",Who is the current president of Notre Dame?,Rev. John Jenkins,"[Architecturally, the school has a Catholic ch...",-1,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.023697998, -0.017766533, 0.06430511, -0.0...","[0.6291710138320923, 0.8571805506944656, 0.829...",320
267,"Architecturally, the school has a Catholic cha...",Mariel Zagunis is notable for winning what?,Olympic gold,"[Architecturally, the school has a Catholic ch...",323,"[[0.048684113, 0.09261659, -0.008530894, 0.011...","[[-0.042852353, 0.070303954, -0.043868415, -0....","[0.8867353349924088, 0.9189914092421532, 0.889...",289


In [62]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

In [63]:
def pred_idx(distances):
    return np.argmin(distances) 

In [64]:
print(accuracy(df_embed["target"], df_embed["predicted_index"]))

0.26765799256505574


In [207]:
df_embed.to_csv("/content/drive/MyDrive/Applied Data Science/project_bart_all-MiniLM-L6-v2", sep='\t')

In [208]:
df_embed.iloc[200,:]

sentence           Architecturally, the school has a Catholic cha...
question           In what year was Father Edward Sorin given two...
text                                                            1842
sentences          [Architecturally, the school has a Catholic ch...
target                                                           241
sent_emb           [[0.048684113, 0.09261659, -0.008530894, 0.011...
quest_emb          [[-0.09045951, -0.014090961, -0.046164166, 0.0...
cosine_sim         [0.7523775547742844, 1.0626248270273209, 0.895...
predicted_index                                                  175
Name: 200, dtype: object

In [65]:
df_1 = df_embed.drop(df_embed.columns[[5, 6,7,8]], axis=1) 
sparkDF= spark.createDataFrame(df_1) 
sparkDF.printSchema()
sp = sparkDF.toPandas()


root
 |-- sentence: string (nullable = true)
 |-- question: string (nullable = true)
 |-- text: string (nullable = true)
 |-- sentences: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- target: long (nullable = true)



In [66]:
sp.to_csv("/content/drive/MyDrive/Applied Data Science/firebase_db", sep='\t')

In [73]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_sm')

In [74]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [76]:
[to_nltk_tree(sent.root).pretty_print()  for sent in en_nlp(df_embed.iloc[0,1]).sents]

                  appear                             
  __________________|____________________________     
 |      |      |    |         |           |      in  
 |      |      |    |         |           |      |    
 |      |      |    To       Mary         in   France
 |      |      |    |      ___|_____      |      |    
did allegedly  ?   whom  the      Virgin 1858 Lourdes



[None]